In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/SNAP'

In [ ]:
!pip install jupyter scikit-learn gitpython peewee fire psycopg2-binary pybtex
!pip install git+https://github.com/brain-score/brainio.git
!pip install git+https://github.com/brain-score/result_caching
!pip install --no-deps git+https://github.com/brain-score/brain-score

In [ ]:
import os
import numpy as np

data_root = './data/'
os.makedirs(data_root, exist_ok=True)

device = 'cuda'

In [ ]:
from dimensionality.wrapper import TorchWrapper
from dimensionality.experiment import Experiment
from dimensionality.brainscore_data import get_neural_data
from dimensionality.regression_utils import regression_metric
import dimensionality.models as models

In [ ]:
region = 'V1'
dataset = 'dicarlo'
image_type = 'original'  # Only matters for Cadena dataset

loader_kwargs = {'batch_size': 200,
                 'shuffle': False,
                 'num_workers': 2,
                 'pin_memory': True,
                 'onehot': True,
                 'labels_from': 'neural_activity'
                 }

data_loader_neural, images, labels = get_neural_data(region=region,
                                                     dataset=dataset,
                                                     image_type=image_type,
                                                     loader_kwargs=loader_kwargs)

# Experiments

In [ ]:
dataset = 'dicarlo'
image_type = 'original'  # Only matters for Cadena dataset

regionNames = ['V1',
               'V2',
               'V4',
               'IT'
               ]

activation_pooling = [None,
                      #   'MaxPool_(1,1)',
                      #   'AvgPool_(1,1)',
                      ]

rand_proj_dim = None
pretrained = {True: 'pretrained',
              False: 'untrained'
              }


modelNames = ['alexnet', 'resnet18',
              # 'resnet50',
              # 'robust_resnet50_l2_3', 'moco_resnet50',
              #   'barlowtwins', 'robust_resnet50_linf_4'
              ]


for region in regionNames:

    data_loader_neural, images, labels = get_neural_data(region=region,
                                                         dataset=dataset,
                                                         image_type=image_type,
                                                         loader_kwargs=loader_kwargs)

    for model_name in modelNames:

        for pooling in activation_pooling:
            for trained in [True, False]:

                data_dir = os.path.join(data_root, f"data_{pooling}_RandProj_{rand_proj_dim}")
                data_fname = os.path.join(data_dir, f"{region}_data_{model_name}_{pretrained[trained]}.npz")
                os.makedirs(data_dir, exist_ok=True)
                print(data_fname)

                # Get the model
                model_kwargs = {'name': model_name,
                                'pretrained': trained,
                                'device': device}
                model, layers, identifier = models.get_model(**model_kwargs)
                model_wrapped = TorchWrapper(model, layers=layers, identifier=identifier, activation_pooling=pooling)

                # Create the Experiment Class and pass additional metrics
                regression_kwargs = {'num_trials': 5,
                                     'reg': 1e-14,
                                     'num_points': 5,
                                     }

                metric_fns = [regression_metric]
                exp = Experiment(model_wrapped, metric_fns=metric_fns, rand_proj_dim=rand_proj_dim)

                # Extract the activations of the layers passed above using data_loader (only uses the inputs)
                exp.get_activations(data_loader_neural())

                # Compute metrics
                metric_kwargs = {'debug': False,
                                 'epsilon': 1e-14
                                 } | regression_kwargs

                exp_metrics = exp.compute_metrics(images=images, labels=labels, **metric_kwargs)
                layers = exp_metrics['layers']

                np.savez(data_fname, exp_metrics=exp_metrics, layers=layers, metric_kwargs=metric_kwargs)